# 리뷰 임베딩

줄거리 임베딩을 제거하고 추출하여 사용자 반응만을 추출

## 설정 및 공통 함수

In [ ]:
import os, re, gc, random, ast
import numpy as np
import pandas as pd
from collections import Counter

import torch
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt

# GPU 환경 안정화
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# ==================== 전역 설정 ====================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# 파일 경로
PATHS = {
    'drama_main': "files/drama/00_drama_main.parquet",
    'movie_main': "files/movie/00_movie_main.parquet",
    'drama_review': "drama_review_final.parquet",
    'movie_review': "movie_review_final.parquet",
}

# 공통 파라미터
CONFIG = {
    'embedding_model': "Qwen/Qwen3-Embedding-0.6B",
    'min_overview_len': 30,
    'top_n_words': 30,
    'review_sample_n': 50_000,
    'review_len_min': 50,
    'review_len_max': 2000,
}

# ==================== 공통 함수 ====================
def clean_text(s: str) -> str:
    """텍스트 클리닝"""
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return ""
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


def create_embedding_model(device="cuda"):
    """임베딩 모델 생성"""
    return SentenceTransformer(CONFIG['embedding_model'], device=device)


def create_bertopic_components(vectorizer_params, umap_params, hdbscan_params, embedding_model=None):
    """BERTopic 컴포넌트 생성"""
    vectorizer = CountVectorizer(**vectorizer_params)
    umap_model = UMAP(**umap_params, random_state=SEED)
    hdbscan_model = HDBSCAN(**hdbscan_params)
    
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer,
        top_n_words=CONFIG['top_n_words'],
        calculate_probabilities=False,
        verbose=True
    )
    
    return topic_model


def extract_keywords_from_topics(topic_model):
    """토픽에서 키워드 추출"""
    keywords_dict = {}
    for tid, tuples in topic_model.get_topics().items():
        if tuples is None:
            continue
        keywords_dict[tid] = [w for w, _ in tuples if w]
    return keywords_dict


def print_topic_stats(df, topic_col, label):
    """토픽 통계 출력"""
    outlier_ratio = (df[topic_col] == -1).mean()
    n_topics_incl = df[topic_col].nunique()
    n_topics_excl = df.loc[df[topic_col] != -1, topic_col].nunique()
    
    print(f"\n{'='*50}")
    print(f"{label} - Topic Statistics")
    print(f"{'='*50}")
    print(f"Outlier(-1) ratio: {outlier_ratio:.4f}")
    print(f"N topics (incl -1): {n_topics_incl}")
    print(f"N topics (excl -1): {n_topics_excl}")
    print(f"\nTop 10 topic sizes:")
    print(df[topic_col].value_counts().head(10))


print("설정 및 공통 함수 로드 완료")

## Step 1: 줄거리 토픽 모델링 (드라마 + 영화)

In [ ]:
def build_plot_topics(data_path, is_drama=True):
    """
    줄거리(overview) 기반 토픽 모델링
    
    Returns:
        df: plot_topic 컬럼이 추가된 데이터프레임
        keywords_dict: {topic_id: [keywords]} 딕셔너리
        topic_info: 토픽 정보 데이터프레임
    """
    # 데이터 로드
    df = pd.read_parquet(data_path).copy()
    
    # 전처리
    df["overview_clean"] = df["overview"].apply(clean_text)
    df = df[df["overview_clean"].str.len() >= CONFIG['min_overview_len']].copy()
    docs = df["overview_clean"].tolist()
    
    n_docs = len(docs)
    label = "DRAMA" if is_drama else "MOVIE"
    print(f"\n{label} - Filtered docs: {n_docs}")
    
    # 임베딩 모델
    embedding_model = create_embedding_model()
    
    # 파라미터 설정 (드라마/영화 구분)
    if is_drama:
        # 드라마: 문서 수가 적으므로 느슨한 설정
        vectorizer_params = {
            'stop_words': 'english',
            'ngram_range': (1, 2),
            'token_pattern': r'\b[a-zA-Z]{3,}\b',
            'min_df': 1,
            'max_df': 1.0
        }
        umap_params = {
            'n_neighbors': 15,
            'n_components': 5,
            'min_dist': 0.0,
            'metric': 'cosine'
        }
        hdbscan_params = {
            'min_cluster_size': 20,
            'min_samples': 5,
            'prediction_data': True
        }
        min_topic_size = 20
    else:
        # 영화: 문서 수가 많으므로 엄격한 설정
        min_df = max(5, int(n_docs * 0.002))
        vectorizer_params = {
            'stop_words': 'english',
            'ngram_range': (1, 2),
            'token_pattern': r'\b[a-zA-Z]{3,}\b',
            'min_df': min_df,
            'max_df': 0.90
        }
        umap_params = {
            'n_neighbors': 20,
            'n_components': 5,
            'min_dist': 0.0,
            'metric': 'cosine'
        }
        hdbscan_params = {
            'min_cluster_size': 50,
            'min_samples': 10,
            'prediction_data': True
        }
        min_topic_size = 50
    
    # BERTopic 생성
    topic_model = create_bertopic_components(
        vectorizer_params, umap_params, hdbscan_params, embedding_model
    )
    topic_model.min_topic_size = min_topic_size
    
    # 학습
    topics, _ = topic_model.fit_transform(docs)
    df["plot_topic"] = topics
    
    # 결과 추출
    keywords_dict = extract_keywords_from_topics(topic_model)
    topic_info = topic_model.get_topic_info()
    
    # 통계 출력
    print_topic_stats(df, "plot_topic", label)
    
    return df, keywords_dict, topic_info


# 실행
print("\n" + "="*50)
print("줄거리 토픽 모델링 시작")
print("="*50)

drama_plot_df, drama_plot_keywords, drama_plot_info = build_plot_topics(
    PATHS['drama_main'], is_drama=True
)

movie_plot_df, movie_plot_keywords, movie_plot_info = build_plot_topics(
    PATHS['movie_main'], is_drama=False
)

print("\n줄거리 토픽 모델링 완료")

## Step 2: 리뷰 토픽 모델링 (드라마 + 영화)

In [ ]:
def create_stopwords_from_plot(plot_keywords_dict, is_drama=True):
    """
    줄거리 키워드를 stopwords로 변환
    
    Args:
        plot_keywords_dict: {topic_id: [keywords]} 딕셔너리
        is_drama: 드라마 여부
    
    Returns:
        combined_stopwords: 통합 stopwords 리스트
    """
    # 줄거리 키워드 수집
    plot_words = set()
    for keywords in plot_keywords_dict.values():
        for w in keywords:
            w = str(w).strip().lower()
            if w:
                plot_words.add(w)
    
    # 범용 서사 단어 제거
    ban_words = {
        "film", "movie", "movies", "story", "stories", "plot",
        "series", "show", "shows", "season", "seasons",
        "episode", "episodes", "character", "characters"
    }
    
    if is_drama:
        ban_words.update({"drama", "dramas", "tv", "television"})
    
    plot_words = plot_words - ban_words
    
    # 리뷰 전용 stopwords
    review_generic_sw = {
        "like", "just", "good", "really", "time", "way",
        "watch", "watched", "watching", "people",
        "dont", "didnt", "doesnt", "isnt", "wasnt", "werent",
        "cant", "couldnt", "wouldnt",
        "im", "ive", "youre", "theyre", "thats", "theres",
        "hes", "shes", "weve", "id",
        "film", "films", "movie", "movies", "show", "shows",
        "series", "season", "seasons", "episode", "episodes",
        "story", "plot", "character", "characters"
    }
    
    if is_drama:
        review_generic_sw.update({"drama", "dramas", "tv", "television"})
    
    # 통합
    base_sw = set(ENGLISH_STOP_WORDS)
    combined = sorted(list(base_sw | plot_words | review_generic_sw))
    
    print(f"Stopwords - ENGLISH: {len(base_sw)}, Plot: {len(plot_words)}, Total: {len(combined)}")
    
    return combined


def build_review_topics(review_path, stopwords, is_drama=True):
    """
    리뷰 기반 토픽 모델링
    
    Args:
        review_path: 리뷰 데이터 경로
        stopwords: stopwords 리스트
        is_drama: 드라마 여부
    
    Returns:
        df_sample: review_topic 컬럼이 추가된 샘플 데이터
        topic_info: 토픽 정보 데이터프레임
    """
    # 데이터 로드
    df = pd.read_parquet(review_path)
    df = df[df['review_text'].notna()].copy()
    df['review_text'] = df['review_text'].astype(str)
    
    # 길이 필터링
    df['__len'] = df['review_text'].str.len()
    df = df[
        (df['__len'] >= CONFIG['review_len_min']) & 
        (df['__len'] <= CONFIG['review_len_max'])
    ].copy()
    
    label = "DRAMA" if is_drama else "MOVIE"
    print(f"\n{label} Review - After filter: {len(df)}")
    
    # 샘플링
    df_sample = df.sample(
        n=min(CONFIG['review_sample_n'], len(df)), 
        random_state=SEED
    ).copy()
    docs = df_sample['review_text'].tolist()
    print(f"{label} Review - Sample docs: {len(docs)}")
    
    # 메모리 정리
    del df
    gc.collect()
    
    # BERTopic 파라미터
    vectorizer_params = {
        'stop_words': stopwords,
        'ngram_range': (1, 2),
        'token_pattern': r'\b[a-zA-Z]{3,}\b',
        'min_df': 20,
        'max_df': 0.95
    }
    
    umap_params = {
        'n_neighbors': 15,
        'n_components': 5,
        'min_dist': 0.0,
        'metric': 'cosine'
    }
    
    hdbscan_params = {
        'min_cluster_size': 50,
        'min_samples': 10,
        'prediction_data': True
    }
    
    # 임베딩 및 학습 (GPU 시도 -> CPU fallback)
    def run_bertopic(device="cuda", batch_size=16):
        embedder = create_embedding_model(device)
        
        with torch.inference_mode():
            embeddings = embedder.encode(
                docs,
                batch_size=batch_size,
                show_progress_bar=True,
                convert_to_numpy=True,
                normalize_embeddings=True
            )
        
        topic_model = create_bertopic_components(
            vectorizer_params, umap_params, hdbscan_params, embedding_model=None
        )
        
        topics, _ = topic_model.fit_transform(docs, embeddings)
        return topic_model, topics
    
    try:
        torch.cuda.empty_cache()
        topic_model, topics = run_bertopic(device="cuda", batch_size=16)
    except Exception as e:
        print(f"\nGPU 실패 -> CPU fallback: {repr(e)}")
        torch.cuda.empty_cache()
        gc.collect()
        topic_model, topics = run_bertopic(device="cpu", batch_size=32)
    
    # 결과 저장
    df_sample['review_topic'] = topics
    df_sample.drop(columns=['__len'], inplace=True, errors='ignore')
    
    topic_info = topic_model.get_topic_info()
    
    # 통계 출력
    print_topic_stats(df_sample, 'review_topic', label + ' Review')
    
    return df_sample, topic_info


# 실행
print("\n" + "="*50)
print("리뷰 토픽 모델링 시작")
print("="*50)

# Stopwords 생성
drama_stopwords = create_stopwords_from_plot(drama_plot_keywords, is_drama=True)
movie_stopwords = create_stopwords_from_plot(movie_plot_keywords, is_drama=False)

# 리뷰 토픽 모델링
drama_review_df, drama_review_info = build_review_topics(
    PATHS['drama_review'], drama_stopwords, is_drama=True
)

movie_review_df, movie_review_info = build_review_topics(
    PATHS['movie_review'], movie_stopwords, is_drama=False
)

print("\n리뷰 토픽 모델링 완료")

## Step 3: 토픽 클러스터링 (영화 + 드라마)

In [ ]:
def parse_representation(x):
    """Representation 컬럼을 list로 변환"""
    if isinstance(x, list):
        return [str(w) for w in x]
    if pd.isna(x):
        return []
    s = str(x).strip()
    try:
        v = ast.literal_eval(s)
        if isinstance(v, list):
            return [str(w) for w in v]
    except Exception:
        pass
    return [w for w in s.replace(",", " ").split() if w]


def make_topic_text(row, topk=20):
    """토픽 대표 텍스트 생성"""
    reps = parse_representation(row.get("Representation", []))
    name = str(row.get("Name", "")).replace("_", " ")
    reps = reps[:topk]
    if reps:
        return " ".join(reps) + " || " + name
    return name


def cluster_keywords(rep_lists, topn=12):
    """클러스터 대표 키워드 추출"""
    c = Counter()
    for reps in rep_lists:
        for w in reps:
            w = str(w).strip().lower()
            if w:
                c[w] += 1
    return [w for w, _ in c.most_common(topn)]


def cluster_topics(info_df, n_clusters=10):
    """
    토픽을 클러스터링
    
    Args:
        info_df: 토픽 정보 데이터프레임
        n_clusters: 클러스터 수
    
    Returns:
        clustered_df: cluster_id가 추가된 데이터프레임
        summary_df: 클러스터 요약 데이터프레임
    """
    df = info_df.copy()
    
    # 컬럼명 통일
    if "review_topic" not in df.columns and "Topic" in df.columns:
        df = df.rename(columns={"Topic": "review_topic"})
    
    # 토픽 텍스트 생성
    df["__topic_text"] = df.apply(make_topic_text, axis=1)
    
    # 임베딩
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedder.encode(
        df["__topic_text"].tolist(),
        show_progress_bar=True,
        normalize_embeddings=True
    )
    
    # 클러스터링
    clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage="average")
    labels = clustering.fit_predict(embeddings)
    df["cluster_id"] = labels
    
    # Representation 파싱
    df["__rep_list"] = df["Representation"].apply(parse_representation) \
        if "Representation" in df.columns else [[]]*len(df)
    
    # 클러스터 요약
    summary_rows = []
    for cid, g in df.groupby("cluster_id"):
        rep_kw = cluster_keywords(g["__rep_list"].tolist(), topn=12)
        
        # 예시 토픽 추출
        if "Count" in g.columns:
            g_sorted = g.sort_values("Count", ascending=False)
        else:
            g_sorted = g
        
        example_cols = ["review_topic", "Name"] if "Name" in g_sorted.columns else ["review_topic"]
        examples = g_sorted.head(3)[example_cols].to_dict("records")
        
        summary_rows.append({
            "cluster_id": int(cid),
            "n_topics": int(len(g)),
            "cluster_keywords": ", ".join(rep_kw),
            "example_topics": examples
        })
    
    summary_df = pd.DataFrame(summary_rows).sort_values(
        "n_topics", ascending=False
    ).reset_index(drop=True)
    
    # 임시 컬럼 제거
    clustered_df = df.drop(columns=["__topic_text", "__rep_list"])
    
    return clustered_df, summary_df


# 실행
print("\n" + "="*50)
print("토픽 클러스터링 시작")
print("="*50)

movie_clustered, movie_summary = cluster_topics(movie_review_info, n_clusters=10)
drama_clustered, drama_summary = cluster_topics(drama_review_info, n_clusters=10)

print(f"\n영화 클러스터: {len(movie_summary)}개")
print(f"드라마 클러스터: {len(drama_summary)}개")
print("\n토픽 클러스터링 완료")

## Step 4: 한글 클러스터명 매핑

In [ ]:
# 한글 클러스터명 매핑
movie_cluster_ko = {
    1: "전반적 완성도·호불호 평가",
    8: "디즈니·애니메이션 감정 반응",
    0: "전쟁·역사 사실성 평가",
    3: "프랜차이즈·액션 콘텐츠 평가",
    7: "음악·퍼포먼스 완성도 평가",
    2: "시즌·홀리데이 감성 반응",
    6: "원작·각색 비교 평가",
    4: "가족·인물 감정 반응",
    5: "사회·인종 이슈 인식 반응",
    9: "종교·신앙 메시지 해석",
}

drama_cluster_ko = {
    0: "전반적 감상·전개 평가",
    2: "세계관·원작 충실도 평가",
    3: "사실성·논쟁 이슈 반응",
    1: "젠더·정체성 표현 반응",
    5: "청소년·교육적 메시지 인식",
    4: "종교·신앙 메시지 해석",
    6: "레이싱·모터스포츠 시청 몰입 반응",
    7: "실험적 연출·예술성 평가",
    8: "요리·경연 리얼리티 반응",
    9: "가족·코미디 정서 반응",
}

# 매핑 적용
movie_clustered["cluster_name_ko"] = movie_clustered["cluster_id"].map(movie_cluster_ko)
drama_clustered["cluster_name_ko"] = drama_clustered["cluster_id"].map(drama_cluster_ko)

movie_summary["cluster_name_ko"] = movie_summary["cluster_id"].map(movie_cluster_ko)
drama_summary["cluster_name_ko"] = drama_summary["cluster_id"].map(drama_cluster_ko)

# 누락 체크
def check_missing_mapping(df, label):
    missing = df[df["cluster_name_ko"].isna()]["cluster_id"].unique().tolist()
    if missing:
        print(f"{label} 매핑 누락 cluster_id: {missing}")
    else:
        print(f"{label} 매핑 완료")

check_missing_mapping(movie_clustered, "영화 info")
check_missing_mapping(drama_clustered, "드라마 info")
check_missing_mapping(movie_summary, "영화 summary")
check_missing_mapping(drama_summary, "드라마 summary")

print("\n한글 클러스터명 매핑 완료")

## Step 5: 결과 저장 (선택사항)

In [ ]:
# 최종 결과만 저장 (중간 파일 없음)
OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 줄거리 토픽
drama_plot_df.to_parquet(f"{OUTPUT_DIR}/drama_with_plot_topic.parquet", index=False)
movie_plot_df.to_parquet(f"{OUTPUT_DIR}/movie_with_plot_topic.parquet", index=False)

# 리뷰 토픽
drama_review_df.to_parquet(f"{OUTPUT_DIR}/drama_review_with_topic.parquet", index=False)
movie_review_df.to_parquet(f"{OUTPUT_DIR}/movie_review_with_topic.parquet", index=False)

# 클러스터 정보 (한글명 포함)
movie_clustered.to_csv(f"{OUTPUT_DIR}/movie_topic_info_clustered.csv", index=False, encoding="utf-8-sig")
drama_clustered.to_csv(f"{OUTPUT_DIR}/drama_topic_info_clustered.csv", index=False, encoding="utf-8-sig")

movie_summary.to_csv(f"{OUTPUT_DIR}/movie_cluster_summary.csv", index=False, encoding="utf-8-sig")
drama_summary.to_csv(f"{OUTPUT_DIR}/drama_cluster_summary.csv", index=False, encoding="utf-8-sig")

print(f"\n모든 결과가 '{OUTPUT_DIR}' 디렉토리에 저장되었습니다.")
print("\n저장된 파일:")
for f in sorted(os.listdir(OUTPUT_DIR)):
    print(f"  - {f}")

## 변수 참조 가이드

### 주요 변수 설명:

**줄거리 관련:**
- `drama_plot_df` / `movie_plot_df`: plot_topic 컬럼이 추가된 원본 데이터
- `drama_plot_keywords` / `movie_plot_keywords`: {topic_id: [keywords]} 딕셔너리
- `drama_plot_info` / `movie_plot_info`: BERTopic의 get_topic_info() 결과

**리뷰 관련:**
- `drama_review_df` / `movie_review_df`: review_topic 컬럼이 추가된 샘플 데이터
- `drama_review_info` / `movie_review_info`: BERTopic의 get_topic_info() 결과

**클러스터 관련:**
- `drama_clustered` / `movie_clustered`: cluster_id와 cluster_name_ko가 추가된 토픽 정보
- `drama_summary` / `movie_summary`: 클러스터별 요약 정보